In [1]:
import sys
# In case you run this without installing the package, you need to add the path to the package

# This is for launching from root folder path
sys.path.append('./ex_fuzzy/')
sys.path.append('./ex_fuzzy/ex_fuzzy/')

# This is for launching from Demos folder
sys.path.append('../ex_fuzzy/')
sys.path.append('../ex_fuzzy/ex_fuzzy/')


import numpy as np

import utils
import fuzzy_sets as t2
import evolutionary_fit as GA
import pandas as pd
import eval_tools
import temporal
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split

In [2]:
def load_occupancy(path='./occupancy_data/', random_mixing=True):
    train_data = pd.read_csv(path + 'datatraining.txt', index_col=0)
    X_train = train_data[['date', 'Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio']]
    y_train = np.squeeze(train_data[['Occupancy']].values)

    test_data = pd.read_csv(path + 'datatest.txt', index_col=0)
    X_test = test_data[['date', 'Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio']]
    y_test = np.squeeze(test_data[['Occupancy']].values)

    if random_mixing:

        X_total = pd.concat([X_train, X_test])
        y_total = np.concatenate([y_train, y_test])

        X_train, X_test, y_train, y_test = train_test_split(X_total, y_total, test_size=0.33, random_state=0)

    return X_train, y_train, X_test, y_test, X_total, y_total


_, _, _, _, X_total, y_total = load_occupancy()
X_total_array = np.array(X_total.drop(columns=['date']))

In [3]:
n_gen = 50
pop_size = 30
nRules = 10
nAnts = 3

fz_type_studied = t2.FUZZY_SETS.temporal
fz_type_inside = t2.FUZZY_SETS.t1
precomputed_partitions = utils.construct_partitions(X_total.drop(columns=['date']), fz_type_inside)

cut_point_morning0 = '00:00:00'
cut_point_morning1 = '10:00:00'
cut_points_morning = [cut_point_morning0, cut_point_morning1]
cut_point_daytime0 = '11:00:00'
cut_point_daytime1 = '19:00:00'
cut_points_daytime = [cut_point_daytime0, cut_point_daytime1]
cut_point_evening0 = '20:00:00'
cut_point_evening1 = '23:00:00'
cutpoints_evening = [cut_point_evening0, cut_point_evening1]


temporal_boolean_markers = utils.temporal_cuts(X_total, cutpoints=[cut_points_morning, cut_points_daytime, cutpoints_evening], time_resolution='hour')
time_moments = np.array([utils.assign_time(a, temporal_boolean_markers) for a in range(X_total.shape[0])])
partitions, partition_markers = utils.temporal_assemble(X_total, y_total, temporal_moments=temporal_boolean_markers)
X_train, X_test, y_train, y_test = partitions
train_markers, test_markers = partition_markers

train_time_moments = np.array([utils.assign_time(a, train_markers) for a in range(X_train.shape[0])])
test_time_moments = np.array([utils.assign_time(a, test_markers) for a in range(X_test.shape[0])])

temp_partitions = utils.create_tempVariables(X_total_array, time_moments, precomputed_partitions) 

X_train = X_train.drop(columns=['date'])
X_test = X_test.drop(columns=['date'])

In [4]:
fl_classifier = temporal.TemporalFuzzyRulesClassifier(nRules=nRules, nAnts=nAnts, 
    linguistic_variables=temp_partitions, n_linguistic_variables=3,
    fuzzy_type=fz_type_studied, verbose=False, tolerance=0.001, n_class=2)
fl_classifier.fit(X_train, y_train, n_gen=n_gen, pop_size=pop_size, time_moments=train_time_moments)

temporal.eval_temporal_fuzzy_model(fl_classifier, X_train, y_train, X_test, y_test, 
                            time_moments=train_time_moments, test_time_moments=test_time_moments,
                            plot_rules=False, print_rules=True, plot_partitions=False)

c:\Users\javi-\anaconda3\envs\datasci\Lib\site-packages\sklearn\metrics\_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
c:\Users\javi-\anaconda3\envs\datasci\Lib\site-packages\sklearn\metrics\_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
c:\Users\javi-\anaconda3\envs\datasci\Lib\site-packages\sklearn\metrics\_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
c:\Users\javi-\anaconda3\envs\datasci\Lib\site-packages\sklearn\metrics\_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_p

ACCURACY
Train performance: 0.9464162408507112
Test performance: 0.9414073451079339
------------
MATTHEW CORRCOEF
Train performance: 0.8737207004919175
Test performance: 0.8635842324647213
------------
MOMENT 0
------------
MATTHEW CORRCOEF
Train performance: 0.956546102674633
Test performance: 0.9378427157827748
------------
MOMENT 1
------------
MATTHEW CORRCOEF
Train performance: 0.7575985548374398
Test performance: 0.7453882026101682
------------
MOMENT 2
------------
MATTHEW CORRCOEF
Train performance: 0.0
Test performance: 0.0
------------
Rules for time step: 0
----------------
Consequent: 0
IF CO2 IS Low WITH DS 0.3231252000350204, ACC 0.9981174698795181
IF Light IS Low AND HumidityRatio IS Low WITH DS 0.17094582743160017, ACC 1.0

Consequent: 1
IF Light IS High WITH DS 0.16375478719683562, ACC 0.9379615952732644

----------------
Rules for time step: 1
----------------
Consequent: 0
IF CO2 IS Low WITH DS 0.0509445134777789, ACC 1.0
IF Humidity IS Medium AND HumidityRatio IS Me

c:\Users\javi-\anaconda3\envs\datasci\Lib\site-packages\sklearn\metrics\_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
c:\Users\javi-\anaconda3\envs\datasci\Lib\site-packages\sklearn\metrics\_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
